# [HW2] Topic Modeling
1. Crawling News
2. Preprocessing
3. Build Term-Document Matrix
4. Topic modeling
5. Visualization

```
🔥 이번 시간에는 Topic Modeling를 직접 크롤링한 뉴스 데이터에 대해서 수행해보는 시간을 갖겠습니다. 

먼저 네이버에서 뉴스 기사를 간단하게 크롤링합니다.
기본적인 전처리 이후 Term-document Matrix를 만들고 이를 non-negative factorization을 이용해 행렬 분해를 하여 Topic modeling을 수행합니다.

t-distributed stochastic neighbor embedding(T-SNE) 기법을 이용해 Topic별 시각화를 진행합니다.
```

In [1]:
!pip install newspaper3k
!pip install konlpy

     |████████████████████████████████| 211 kB 4.4 MB/s 
     |████████████████████████████████| 87 kB 6.3 MB/s 
     |████████████████████████████████| 7.4 MB 47.3 MB/s 
     |████████████████████████████████| 81 kB 9.4 MB/s 
  Created wheel for tinysegmenter: filename=tinysegmenter-0.3-py3-none-any.whl size=13553 sha256=cfcc0ce031c3fd6d9a6f8b43801c123beb84166c2b246795ba4b5c111b49f64e
  Stored in directory: /root/.cache/pip/wheels/df/67/41/faca10fa501ca010be41b49d40360c2959e1c4f09bcbfa37fa
  Created wheel for feedfinder2: filename=feedfinder2-0.0.4-py3-none-any.whl size=3357 sha256=eba8356e8c1d1eb46aa9800b8f643c7c275e64bf89a8c6b29a47547afa742721
  Stored in directory: /root/.cache/pip/wheels/7f/d4/8f/6e2ca54744c9d7292d88ddb8d42876bcdab5e6d84a21c10346
  Created wheel for jieba3k: filename=jieba3k-0.35.1-py3-none-any.whl size=7398404 sha256=54c1ce103760e69f8df36a6c91d78f167f72dc6db1124947c91ea5fedededde1
  Stored in directory: /root/.cache/pip/wheels/4c/91/46/3c208287b726df325a597957432

In [2]:
# 크롤링에 필요한 패키지 설치
from bs4 import BeautifulSoup
from newspaper import Article
from time import sleep
from time import time
from dateutil.relativedelta import relativedelta
from datetime import datetime
from multiprocessing import Pool
import json
import requests
import re
import sys

```
💡 Crawling(크롤링)이란?

크롤링은 웹 페이지에서 필요한 데이터를 추출해내는 작업을 말합니다.
이번 시간에는 정적 페이지인 네이버의 뉴스 신문 기사 웹페이지를 크롤링합니다.

HTML은 설명되어 있는 자료가 많기 때문에 생략하도록 하겠습니다.
HTML 구조 파악 및 태그에 대한 설명은 아래 참고자료를 살펴봐주세요 !
```

참고: [위키피디아: 정적페이지](https://ko.wikipedia.org/wiki/%EC%A0%95%EC%A0%81_%EC%9B%B9_%ED%8E%98%EC%9D%B4%EC%A7%80)

참고: [생활코딩: HTML](https://opentutorials.org/course/2039)

In [3]:
def crawl_news(query: str=None, crawl_num: int=1000, workers: int=4):
    '''뉴스 기사 텍스트가 담긴 list를 반환합니다.

    Keyword arguments:
    query -- 검색어 (default None)
    crawl_num -- 수집할 뉴스 기사의 개수 (defualt 1000)
    workers -- multi-processing시 사용할 thread의 개수 (default 4)
    '''

    url = 'https://search.naver.com/search.naver?where=news&sm=tab_jum&query={}'
    articleList = []
    crawled_url = set()
    keyboard_interrupt = False
    t = time()
    idx = 0
    page = 1

    
    # 서버에 url 요청의 결과를 선언
    res = requests.get(url.format(query))
    sleep(0.5)
    # res를 parsing할 parser를 선언
    bs = BeautifulSoup(res.text, 'html.parser')
    
    with Pool(workers) as p:
        while idx < crawl_num:            
            table = bs.find('ul', {'class': 'list_news'})
            li_list = table.find_all('li', {'id': re.compile('sp_nws.*')})
            area_list = [li.find('div', {'class':'news_area'}) for li in li_list]
            a_list = [area.find('a', {'class':'news_tit'}) for area in area_list]
            
            for n in a_list[:min(len(a_list), crawl_num-idx)]:
                articleList.append(n.get('title'))
                idx += 1
            page += 1

            pages = bs.find('div', {'class': 'sc_page_inner'})
            next_page_url = [p for p in pages.find_all('a') if p.text == str(page)][0].get('href')

            req = requests.get('https://search.naver.com/search.naver' + next_page_url)
            bs = BeautifulSoup(req.text, 'html.parser')
    return articleList

```
🔥 이제 '구글'이라는 이름으로 뉴스 기사 1000개의 제목을 크롤링하겠습니다.
```

In [4]:
query = '구글'

articleList = crawl_news(query)

In [5]:
articleList[:10]

['웹젠 ‘뮤오리진3’, 구글 매출 탑5 ‘또 통했다’',
 '원스토어 앱, 이제 구글 안드로이드 오토서도 작동된다',
 '구글까지 우크라 돕는다…"실시간 도로 상황 비공개 전환"',
 '구글, 유럽 유튜브서 러시아 국영방송·통신사 채널 차단',
 '방탄소년단 정국, 2021년 美구글에서 가장 많이 검색된 K팝 남자 아이돌 1위',
 '웹젠 뮤오리진3, 구글 매출 톱5...출시 일주일만',
 '구글, 우크라 지도 서비스 일시 중단…"현지 안전 고려"',
 '[DBR/Management Solution]‘스스로 목표 설정’ OKR, 구글 키운 핵심 비결',
 "구글, '창구 프로그램' 4기 개발사 모집",
 '애플부터 구글, 포드까지…러시아에 등 돌리는 (주) 미국']

```
🔥 태거(tagger)를 이용해 한글 명사와 알파벳만을 추출해서 term-document matrix (tdm)을 만들겠습니다.

태거(tagger)는 tokenization에서 조금 더 자세히 다루도록 하겠습니다.
```

참고: [konlpy: morph analyzer](https://konlpy-ko.readthedocs.io/ko/v0.4.3/morph/)

## Preprocessing

In [7]:
from konlpy.tag import Okt
from collections import Counter
import json

# Okt 형태소 분석기 선언
t = Okt()

words_list_ = []
vocab = Counter()
tag_set = set(['Noun', 'Alpha'])
stopwords = set(['글자'])

for i, article in enumerate(articleList):
    if i % 100 == 0:
        print(i)
    
    # tagger를 이용한 품사 태깅
    words = t.pos(article, norm=True, stem=True)
    
    ############################ ANSWER HERE ################################
    # TODO: 다음의 조건을 만족하는 단어의 리스트를 완성하세요.
    # 조건 1: 명사와 알파벳 tag를 가진 단어
    # 조건 2: 철자 길이가 2이상인 단어 
    # 조건 3: stopwords에 포함되지 않는 단어
    words = [w for w,t in words if t in tag_set and len(w) > 1 and w not in stopwords]
    #########################################################################        

    vocab.update(words)
    words_list_.append((words, article))
    
vocab = sorted([w for w, freq in vocab.most_common(10000)])
word2id = {w: i for i, w in enumerate(vocab)}
words_list = []
for words, article in words_list_:
    words = [w for w in words if w in word2id]
    if len(words) > 10:
        words_list.append((words, article))
        
del words_list_

0
100
200
300
400
500
600
700
800
900


## Build document-term matrix

```
🔥 이제 document-term matrix를 만들어보겠습니다.
document-term matrix는 (문서 개수 x 단어 개수)의 Matrix입니다.
```

참고: [Document-Term Matrix](https://wikidocs.net/24559)

In [11]:
from sklearn.feature_extraction.text import TfidfTransformer
import numpy as np

dtm = np.zeros((len(words_list), len(vocab)), dtype=np.float32)
print(dtm.shape)
for i, (words, article) in enumerate(words_list):
    for word in words:
        dtm[i, word2id[word]] += 1
print('dtm : ', dtm)       
dtm = TfidfTransformer().fit_transform(dtm)

(63, 1398)
dtm :  [[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


```
🔥 document-term matrix를 non-negative factorization(NMF)을 이용해 행렬 분해를 해보겠습니다.

💡 Non-negative Factorization이란?

NMF는 주어진 행렬 non-negative matrix X를 non-negative matrix W와 H로 행렬 분해하는 알고리즘입니다.
이어지는 코드를 통해 W와 H의 의미에 대해 파악해봅시다.
```
참고: [Non-negative Matrix Factorization](https://angeloyeo.github.io/2020/10/15/NMF.html)

## Topic modeling

In [12]:
# Non-negative Matrix Factorization
from sklearn.decomposition import NMF

K=5
nmf = NMF(n_components=K, alpha=0.1)

```
🔥 sklearn의 NMF를 이용해 W와 H matrix를 구해봅시다.
W는 document length x K, H는 K x term length의 차원을 갖고 있습니다.
W의 하나의 row는 각각의 feature에 얼만큼의 가중치를 줄 지에 대한 weight입니다.
H의 하나의 row는 하나의 feature를 나타냅니다.

우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [13]:
W = nmf.fit_transform(dtm)
H = nmf.components_

/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:1425: FutureWarning: `alpha` was deprecated in version 1.0 and will be removed in 1.2. Use `alpha_W` and `alpha_H` instead
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/decomposition/_nmf.py:294: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  FutureWarning,


```
🔥 우선 하나의 Topic (H의 n번째 row)에 접근해서 해당 topic에 대해 값이 가장 높은 20개의 단어를 출력해보겠습니다.
```

In [14]:
for k in range(K):
    print(f"{k}th topic")
    for index in H[k].argsort()[::-1][:20]:
        print(vocab[index], end=' ')
    print()

0th topic
탐지 위협 화폐 프리뷰 퍼블릭 암호 클라우드 채굴 VMTD 발표 버전 구글 멀웨어 공개 기능 가상머신 애플 AI 보호 개인정보 
1th topic
기대 클로버 협업 전략 유저 관계 구축 확대 글로벌 게임 구글 AI LG 거대 우리 동맹 생태계 결성 지역 가능 
2th topic
개발 모집 중소 진출 창구 프로그램 국내 해외 성장 지원 구글플레이 AI 스타트업 구글 개사 트래픽 대리인 과기부 이상 지정 
3th topic
정식 신마 삼국 세계 대결 출시 MMORPG 모바일 플레이 XiaoMa 구글 신작 우마 재미 가지 육성 무장 국내 구글플레이 애플 
4th topic
체결 네이버웹툰 웹툰 협약 업무 게임 구글플레이 진원 IP 제작 진흥 콘텐츠 한국 글로벌 기반 해외진출 MOU 플레이 구글 출신 


```
🔥 이번에는 W에서 하나의 Topic (W의 n번째 column)에 접근해서 해당 topic에 대해 값이 가장 높은 3개의 뉴스 기사 제목을 출력해보겠습니다.
```

In [15]:
for k in range(K):
    print(f"==={k}th topic===")
    for index in W[:, k].argsort()[::-1][:3]:
        print(words_list[index][1])
    print('\n')

===0th topic===
"암호화폐 채굴 위협 탐지"… 구글 클라우드, 'VMTD' 퍼블릭 프리뷰 버전 발표
구글 클라우드, 암호화폐 채굴 위협 탐지하는 VMTD 퍼블릭 프리뷰 버전 발표
구글 클라우드, 암호화폐 채굴 위협 탐지하는 'VMTD' 퍼블릭 프리뷰 버전 발표


===1th topic===
클로버게임즈, 구글과 전략적 협업 관계 구축…글로벌 유저 확대 기대
클로버게임즈, 구글과 전략적 협업 관계 구축 … 글로벌 유저 확대 기대
클로버게임즈, 구글과 전략적 협업 관계 구축…글로벌 유저 확대 기대


===2th topic===
구글플레이, 국내 중소 개발사 성장과 해외 진출 지원...'창구 프로그램 4기' 개발사 모집
구글플레이, 국내 중소 개발사 성장과 해외 진출 지원하는 ‘창구 프로그램 4기’ 개발사 모집
구글플레이, 국내 중소 개발사 성장과 해외 진출 지원하는 ‘창구 프로그램 4기’ 개발사 모집


===3th topic===
XiaoMa, 모바일 MMORPG ‘뉴 삼국세계:신마 대결’ 구글 플레이 정식 출시
쇼우마, 모바일 MMORPG ‘뉴 삼국세계:신마 대결’ 구글 플레이 정식 출시
신작 모바일 MMORPG ‘뉴 삼국세계:신마 대결’ 구글 플레이 정식 출시


===4th topic===
네이버웹툰, 구글플레이 및 한국콘텐츠진흥원과 '글로벌 웹툰 게임스' 업무협약 체결
콘진원-구글플레이-네이버웹툰, 웹툰 IP 기반 게임 제작 위한 업무협약 체결
콘진원·구글플레이·네이버웹툰, 웹툰 IP게임 제작 및 해외진출 업무협약 체결




```
❓ 2번째 토픽에 대해 가장 높은 가중치를 갖는 제목 5개를 출력해볼까요?
```

In [16]:
#TODO
index_list = W[:, 2].argsort()[::-1][:5]
for index in index_list:
    print(words_list[index][1])

구글플레이, 국내 중소 개발사 성장과 해외 진출 지원...'창구 프로그램 4기' 개발사 모집
구글플레이, 국내 중소 개발사 성장과 해외 진출 지원하는 ‘창구 프로그램 4기’ 개발사 모집
구글플레이, 국내 중소 개발사 성장과 해외 진출 지원하는 ‘창구 프로그램 4기’ 개발사 모집
구글플레이, 국내 중소 개발사 성장과 해외 진출 지원하는 ‘창구 프로그램 4기’ 개발사 모집
과기부, '국내 트래픽 1% 이상' 5개사 지정…구글·메타 '국내 대리인' 의무화


```
🔥 이번에는 t-SNE를 이용해 Topic별 시각화를 진행해보겠습니다.

💡 t-SNE는 무엇인가요?

t-Stochastic Neighbor Embedding(t-SNE)은 고차원의 벡터를 
저차원(2~3차원) 벡터로 데이터간 구조적 특징을 유지하며 축소를 하는 방법 중 하나입니다.

주로 고차원 데이터의 시각화를 위해 사용됩니다.
```

참고: [lovit: t-SNE](https://lovit.github.io/nlp/representation/2018/09/28/tsne/#:~:text=t%2DSNE%20%EB%8A%94%20%EA%B3%A0%EC%B0%A8%EC%9B%90%EC%9D%98,%EC%9D%98%20%EC%A7%80%EB%8F%84%EB%A1%9C%20%ED%91%9C%ED%98%84%ED%95%A9%EB%8B%88%EB%8B%A4.)

참고: [ratsgo: t-SNE](https://ratsgo.github.io/machine%20learning/2017/04/28/tSNE/)

## Visualization

In [17]:
from sklearn.manifold import TSNE

# n_components = 차원 수
tsne = TSNE(n_components=2, init='pca', verbose=1)

# W matrix에 대해 t-sne를 수행합니다.
W2d = tsne.fit_transform(W)

# 각 뉴스 기사 제목마다 가중치가 가장 높은 topic을 저장합니다.
topicIndex = [v.argmax() for v in W]

/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:793: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/sklearn/manifold/_t_sne.py:986: FutureWarning: The PCA initialization in TSNE will change to have the standard deviation of PC1 equal to 1e-4 in 1.2. This will ensure better convergence.
  FutureWarning,


[t-SNE] Computing 62 nearest neighbors...
[t-SNE] Indexed 63 samples in 0.000s...
[t-SNE] Computed neighbors for 63 samples in 0.001s...
[t-SNE] Computed conditional probabilities for sample 63 / 63
[t-SNE] Mean sigma: 0.097801
[t-SNE] KL divergence after 250 iterations with early exaggeration: 49.678047
[t-SNE] KL divergence after 1000 iterations: -0.468481


In [18]:
from bokeh.models import HoverTool
from bokeh.palettes import Category20
from bokeh.io import show, output_notebook
from bokeh.plotting import figure, ColumnDataSource
output_notebook()

# 사용할 툴들
tools_to_show = 'hover,box_zoom,pan,save,reset,wheel_zoom'
p = figure(plot_width=720, plot_height=580, tools=tools_to_show)

source = ColumnDataSource(data={
    'x': W2d[:, 0],
    'y': W2d[:, 1],
    'id': [i for i in range(W.shape[0])],
    'document': [article for words, article in words_list],
    'topic': [str(i) for i in topicIndex],  # 토픽 번호
    'color': [Category20[K][i] for i in topicIndex]
})
p.circle(
    'x', 'y',
    source=source,
    legend='topic',
    color='color'
)

# interaction
p.legend.location = "top_left"
hover = p.select({'type': HoverTool})
hover.tooltips = [("Topic", "@topic"), ('id', '@id'), ("Article", "@document")]
hover.mode = 'mouse'

show(p)